In [1]:
#this also calculates Overall Avg transition matrix
#on 2/26, going to add some logic to see if we can reduce the time it takes to make a random choice given distribution
#2/29 going to expand quartile to allow for longer play possibility
from helper_functions.clean_transitions import clean_transition_column
import pandas as pd
from itertools import product
from matplotlib import pyplot as plt

In [2]:
transition_times = pd.read_excel('All_Transition_With_Times_Not_Aggregated_v02.xlsx')
# c1=transition_times.copy()

In [3]:
transition_times['Transition'] = clean_transition_column(transition_times['Transition'])

In [4]:
valid_transitions = pd.read_excel('ALL_VALID_TRANSITIONS_v03.xlsx')

In [5]:
valid_transitions['Transition']=clean_transition_column(valid_transitions['Transition'])

In [6]:
unique_valid_trans = valid_transitions['Transition'].unique()

In [7]:
#we have a list of all valid transitions. Going to get a list of all transitions including invalid ones just so we can assign 0 score to invalid ones (since they happen in real games)
possible_states = ['Ai0',
'Ai1',
'Ai2',
'Ai3',
'Ar0',
'Af0',
'Af1',
'Af2',
'Af3',
'Bi0',
'Bi1',
'Bi2',
'Bi3',
'Br0',
'Bf0',
'Bf1',
'Bf2',
'Bf3']

possible_transitions = list(product(possible_states,possible_states))

In [8]:
#some basic ground rules - any F to F transition should be 0 seconds
unique_trans = transition_times['Transition'].unique()
for t in unique_trans:
    if (('f' in t[0]) & ('f' in t[1]))|(t not in list(unique_valid_trans)):
        transition_times.loc[transition_times['Transition'] == t, 'Time'] = 0


In [9]:
transition_times

,Transition,Time
0,"(Bi0, Ai3)",24
1,"(Ai3, Ar0)",32
2,"(Ar0, Ar0)",22
3,"(Ar0, Br0)",0
4,"(Br0, Ar0)",26
...,...,...
810192,"(Bf1, Ar0)",3
810193,"(Ar0, Br0)",27
810194,"(Br0, Ai3)",6
810195,"(Ai3, Bi2)",30


In [10]:
#find interquartile range of each transition's time durations
quartiles = transition_times.groupby('Transition')['Time'].quantile([0.1, 0.9]).unstack()

In [11]:
#merge original DF with interquartile range df to filter
df_merged = pd.merge(transition_times, quartiles, left_on='Transition', right_index=True, suffixes=('', '_quartile'))

filtered_df = df_merged[(df_merged['Time'] >= df_merged[0.1]) & (df_merged['Time'] <= df_merged[0.9])]


In [12]:
smaller_ranged_times = filtered_df[['Transition','Time']].copy()

In [13]:
group_aggs = smaller_ranged_times.groupby('Transition').agg(['mean','std','count']).reset_index()

In [14]:
pd.DataFrame(group_aggs.values, columns = ['Transition', 'Avg_Time','std','count'])

,Transition,Avg_Time,std,count
0,"(Af0, Af0)",0.0,0.0,8625
1,"(Af0, Af1)",0.0,0.0,24602
2,"(Af0, Af2)",0.0,0.0,4
3,"(Af0, Af3)",0.0,NaN,1
4,"(Af0, Ai0)",0.0,0.0,40
...,...,...,...,...
275,"(Br0, Bi0)",4.438616,5.211636,10752
276,"(Br0, Bi1)",0.0,0.0,9
277,"(Br0, Bi2)",0.0,0.0,286
278,"(Br0, Bi3)",0.0,0.0,123


In [15]:
#we have reduced dataframe for times but we want to use original for probabilities/frequencies.
#this is incomplete as in not all possible state combinations are represented, but that should be fine since only VALID transitions should be called
#switch to v04 on 2/29
smaller_ranged_times.to_excel('Transition_times_reduced_v03.xlsx',index=False)

In [16]:
transitions_agg = valid_transitions.groupby('Transition')['Period'].count().reset_index()

In [17]:
for t in possible_transitions:
    if t not in list(transitions_agg['Transition'].values):
        curr_row = pd.DataFrame([[t, 0]], columns=['Transition','Period'])
        transitions_agg=pd.concat([transitions_agg,curr_row])

In [18]:
transitions_agg = transitions_agg.rename(columns={'Period':'Count'})

In [19]:
transitions_agg['Starting_State'] = [x[0] for x in transitions_agg['Transition']]
transitions_agg['Ending_State'] = [x[1] for x in transitions_agg['Transition']]

In [20]:
transitions_agg

,Transition,Count,Starting_State,Ending_State
0,"(Af0, Af0)",11071,Af0,Af0
1,"(Af0, Af1)",30975,Af0,Af1
2,"(Af0, Ar0)",1043,Af0,Ar0
3,"(Af0, Bi0)",17,Af0,Bi0
4,"(Af0, Bi1)",8128,Af0,Bi1
...,...,...,...,...
0,"(Bf3, Bi3)",0,Bf3,Bi3
0,"(Bf3, Bf0)",0,Bf3,Bf0
0,"(Bf3, Bf1)",0,Bf3,Bf1
0,"(Bf3, Bf2)",0,Bf3,Bf2


In [21]:
transition_counts = transitions_agg.groupby(['Starting_State', 'Ending_State'])['Count'].sum().reset_index()


In [22]:
transition_matrix = transition_counts.pivot(index='Starting_State', columns='Ending_State', values='Count').fillna(0)

In [23]:
transition_matrix

Ending_State,Af0,Af1,Af2,Af3,Ai0,Ai1,Ai2,Ai3,Ar0,Bf0,Bf1,Bf2,Bf3,Bi0,Bi1,Bi2,Bi3,Br0
Starting_State,,,,,,,,,,,,,,,,,,
Af0,11071,30975,0,0,0,0,0,0,1043,0,0,0,0,17,8128,0,0,4939
Af1,310,873,0,0,0,0,0,0,1188,0,0,0,0,5,24093,0,0,5911
Af2,0,0,0,0,0,0,0,0,491,0,0,0,0,18,5672,0,0,2225
Af3,0,0,0,0,0,0,0,0,33,0,0,0,0,3,243,0,0,77
Ai0,2532,0,514,38,2331,0,0,0,10149,0,0,0,0,798,0,6277,2860,12731
Ai1,3612,0,669,37,1607,0,0,0,4232,0,0,0,0,601,0,5442,2568,11733
Ai2,7673,0,1490,80,4455,0,0,0,8965,0,0,0,0,1368,0,14180,7485,29680
Ai3,3342,0,600,39,2003,0,0,0,3810,0,0,0,0,601,0,6118,3185,12671
Ar0,26829,0,5329,220,15685,0,0,0,29302,0,0,0,0,3968,0,51727,20275,80597


In [24]:
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)

In [25]:
transition_matrix.to_excel('avg_transition_matrix.xlsx')

In [16]:
#switch to v04 on 2/29
smaller_ranged_times = pd.read_excel('Transition_times_reduced_v02.xlsx')

In [17]:
smaller_ranged_times['Transition']=clean_transition_column(smaller_ranged_times['Transition'])

In [18]:
smaller_ranged_times

,Transition,Time
0,"(Bi0, Ai3)",8
1,"(Bi0, Ai3)",10
2,"(Bi0, Ai3)",7
3,"(Bi0, Ai3)",9
4,"(Bi0, Ai3)",14
...,...,...
511945,"(Bi1, Af3)",0
511946,"(Af2, Bi3)",0
511947,"(Af1, Ai3)",0
511948,"(Af3, Ai1)",0


In [19]:
smaller_ranged_times.groupby(['Transition','Time']).count()

Empty DataFrame
Columns: []
Index: [((Af0, Af0), 0), ((Af0, Af1), 0), ((Af0, Af2), 0), ((Af0, Af3), 0), ((Af0, Ai0), 0), ((Af0, Ai1), 0), ((Af0, Ai2), 0), ((Af0, Ai3), 0), ((Af0, Ar0), 0), ((Af0, Ar0), 1), ((Af0, Ar0), 2), ((Af0, Bf0), 0), ((Af0, Bf1), 0), ((Af0, Bi0), 0), ((Af0, Bi0), 1), ((Af0, Bi0), 2), ((Af0, Bi0), 8), ((Af0, Bi0), 10), ((Af0, Bi1), 0), ((Af0, Bi2), 0), ((Af0, Bi3), 0), ((Af0, Br0), 0), ((Af0, Br0), 1), ((Af0, Br0), 2), ((Af1, Af0), 0), ((Af1, Af1), 0), ((Af1, Ai0), 0), ((Af1, Ai1), 0), ((Af1, Ai3), 0), ((Af1, Ar0), 0), ((Af1, Ar0), 1), ((Af1, Ar0), 2), ((Af1, Bf0), 0), ((Af1, Bf1), 0), ((Af1, Bf2), 0), ((Af1, Bi0), 0), ((Af1, Bi1), 0), ((Af1, Bi2), 0), ((Af1, Br0), 0), ((Af1, Br0), 1), ((Af1, Br0), 2), ((Af2, Af0), 0), ((Af2, Af1), 0), ((Af2, Ai0), 0), ((Af2, Ai1), 0), ((Af2, Ai2), 0), ((Af2, Ai3), 0), ((Af2, Ar0), 0), ((Af2, Ar0), 1), ((Af2, Ar0), 2), ((Af2, Bf0), 0), ((Af2, Bf1), 0), ((Af2, Bf2), 0), ((Af2, Bi0), 0), ((Af2, Bi1), 0), ((Af2, Bi2), 0), ((Af2, Bi3), 0), ((Af2, Br0), 1), ((Af2, Br0), 2), ((Af3, Af0), 0), ((Af3, Af1), 0), ((Af3, Ai0), 0), ((Af3, Ai1), 0), ((Af3, Ai2), 0), ((Af3, Ar0), 0), ((Af3, Ar0), 1), ((Af3, Ar0), 2), ((Af3, Bf0), 0), ((Af3, Bf1), 0), ((Af3, Bi0), 7), ((Af3, Bi1), 0), ((Af3, Bi1), 1), ((Af3, Bi2), 0), ((Af3, Br0), 1), ((Af3, Br0), 2), ((Af3, Br0), 3), ((Ai0, Af0), 2), ((Ai0, Af0), 3), ((Ai0, Af0), 4), ((Ai0, Af0), 5), ((Ai0, Af0), 6), ((Ai0, Af0), 7), ((Ai0, Af0), 8), ((Ai0, Af0), 9), ((Ai0, Af0), 10), ((Ai0, Af0), 11), ((Ai0, Af0), 12), ((Ai0, Af0), 13), ((Ai0, Af1), 0), ((Ai0, Af2), 6), ((Ai0, Af2), 7), ((Ai0, Af2), 8), ((Ai0, Af2), 9), ((Ai0, Af2), 10), ((Ai0, Af2), 11), ((Ai0, Af2), 12), ((Ai0, Af2), 13), ((Ai0, Af2), 14), ((Ai0, Af3), 5), ((Ai0, Af3), 6), ...]

[1241 rows x 0 columns]

In [20]:
from tqdm import tqdm
import numpy as np

In [21]:
new_df = pd.DataFrame()
for t in tqdm(smaller_ranged_times['Transition'].unique()):
    temp_df = smaller_ranged_times[smaller_ranged_times['Transition']==t].groupby('Time').count().reset_index()
    temp_df['Freq'] = temp_df['Transition']/temp_df['Transition'].sum()
    temp_df['Transition']=str(t)
    temp_df=temp_df[['Transition','Time','Freq']].copy()
    new_df=pd.concat([new_df, temp_df]).reset_index(drop=True)

  0%|          | 0/280 [00:00<?, ?it/s]

100%|██████████| 280/280 [00:16<00:00, 16.89it/s]


In [22]:
new_df['Transition']=clean_transition_column(new_df['Transition'])

In [23]:
#switch to v05 on 2/29
new_df.to_excel('Transition_times_reduced_v05.xlsx')